In [1]:
#%%writefile get_ruten.py
#%load get_bookprice.py
#%run get_bookprice.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
from django.db.models import Q
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq
#
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
#
import requests
import urllib.parse
import pandas as pd
import numpy as np
import random
import re
import json
import csv
#
from get_proxy import get_proxy
#from get_mollie import get_mollie
#
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'wtb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
#from mainsite.models import Bookinfo,Bookprice,Store
#from dict_stores import url_qs

#
def get_bp(isbn):
    #url_api1="https://rtapi.ruten.com.tw/api/search/v2/index.php/core/prod?sort=prc%2Fac&q="
    #url_api2="https://rtapi.ruten.com.tw/api/search/v2/index.php/m/core/prod?sort=prc%2Fac&q="
    #url_apis=[url_api1,url_api2]
    url_q="http://www.readingtimes.com.tw/ReadingTimes/site/q__"+isbn+"/414/default.aspx"
    #url_q='http://www.readingtimes.com.tw/ReadingTimes/site/q__9789571380049/414/default.aspx'
    #(1)先抓價格最低的店內碼______________________
    #
    fake_header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )    
    UA=fake_header.generate()  
    ippo=get_proxy(which="OK",now=True)
    proxies={
            "http": "http://"+ippo,
            #"https": "http://"+ippo
            }
    try:
        #requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'TLS_RSA_WITH_3DES_EDE_CBC_SHA DES-CBC3-SHA'
        r = requests.get(url_q, 
                         headers=UA,
                         #proxies=proxies,
                         #allow_redirects=False,
                         verify=False,
                         timeout=60)    
        r.encoding='utf8' 
        doc=pq(r.text)
        #print(r.text)
        r.close()
        table=doc.find("div:Contains('商品名稱')").parent().parent()
        if table:
            tmp=table.find('td.etext8red')
            url_book  =tmp.eq(0).parent().find('a').eq(0).attr('href') or ''
            price_sale=tmp.eq(1).text() or ''
            #
            return url_book,price_sale
        return 'notable'
    except Exception as err:
        return str(err)
    
        


In [2]:
isbn='4712966622177'
%time get_bp(isbn)

CPU times: user 12.7 ms, sys: 2.12 ms, total: 14.9 ms
Wall time: 1.57 s


('http://www.readingtimes.com.tw/ReadingTimes/ProductPage.aspx?gp=productdetail&cid=mcac(SellItems)&id=1AY1033',
 '474')